# WIP: Back-test

This example is still WIP.
* Load Data
* Back Test - the Basic Idea

In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-04 21:21:45,164 INFO Keys of d_instrument_data: dict_keys(['EEMA', 'IEV', 'IVV', 'VPL'])
2022-03-04 21:21:45,169 INFO IVV (SP500 ETF): 
        date       open       high        low      close  close_adj   volume
0 2000-05-19  142.65625  142.65625  140.25000  140.68750  94.121216   775500
1 2000-05-22  140.59375  140.59375  136.81250  139.81250  93.535789  1850600
2 2000-05-23  140.21875  140.21875  137.68750  137.68750  92.114151   373900
3 2000-05-24  137.75000  140.06250  136.65625  139.75000  93.494003   400300
4 2000-05-25  140.03125  140.93750  137.87500  138.46875  92.636810    69600



### Back Test - the Basic Idea

WIP
* back test is to understand how your strategy behaves under various scenanios, not to prove your strategy can beat xxx.
* most investment products are not built to beat xxx.
* historical performance cannot represent future performance.


### Back Test - WIP

In [21]:
price_col = 'close_adj'
end_date = None # '2022-01-31'
backtest_years = 10
minimum_training_years = 5
maximum_training_years = 10
verbose = 2
printfunc = print

import pandas as pd


In [22]:
if end_date is None:
    backtest_end_date = None
    for ticker, df_price in d_instrument_data.items():
        if backtest_end_date is None:
            backtest_end_date = df_price['date'].max()
        else:
            backtest_end_date = max(backtest_end_date, df_price['date'].max())
else:
    backtest_end_date = pd.to_datetime(end_date)

backtest_start_date = backtest_end_date - pd.DateOffset(years=backtest_years)
if verbose > 0: printfunc(f"To backtest {backtest_years} years, from {backtest_start_date.date()} to {backtest_end_date.date()}")

To backtest 10 years, from 2012-02-28 to 2022-02-28


In [29]:
max_start_date = backtest_start_date - pd.DateOffset(years=minimum_training_years)
min_start_date = backtest_start_date - pd.DateOffset(years=maximum_training_years)

min_start_date, max_start_date

(Timestamp('2002-02-28 00:00:00'), Timestamp('2007-02-28 00:00:00'))

In [28]:
df_instrument_prices = pd.DataFrame()
for ticker, df_price in d_instrument_data.items():
    if df_price['date'].min() > max_start_date:
        printfunc(f"{ticker} start date {df_price['date'].min().date()} is beyond {max_start_date.date()} for full backtest.")
    
    df = df_price[df_price['date']>=max_start_date][['date',price_col]].reset_index(drop=True)
    df.rename(columns={price_col:'price_'+ticker}, inplace=True)

    if len(df_instrument_prices)==0:
        df_instrument_prices = df.copy()
        continue
    else:
        df_instrument_prices = df_instrument_prices.merge(df, 'outer', 'date')

df_instrument_prices = df_instrument_prices.sort_values('date').reset_index(drop=True)
df_instrument_prices.fillna(method='pad', inplace=True)
df_instrument_prices.index = df_instrument_prices['date']

df_instrument_prices.head()

EEMA start date 2012-02-09 is beyond 2007-02-28 for full backtest.


,date,price_EEMA,price_IEV,price_IVV,price_VPL
date,,,,,
2007-02-28,2007-02-28,NaN,33.662262,105.162704,45.376842
2007-03-01,2007-03-01,NaN,33.219002,104.790573,45.079662
2007-03-02,2007-03-02,NaN,32.785374,103.503021,44.313572
2007-03-05,2007-03-05,NaN,32.120483,102.416382,43.580528
2007-03-06,2007-03-06,NaN,33.007000,104.217491,44.874924


In [26]:
df_instrument_prices.rolling()